# ITR Tool - Template Temperature Score Calculation

This notebook provides a simple example of the ITR Toolkit as a means to evaluate Portfolio Alignment.  Text boxes (called Markdown cells) like this provide some explanation, and then code cells either do the work just explained or will do work that will be explained.

All data needed to run this demonstration should be local and no acccess to the internet should be required.  However, if you want to delete a file and re-download it (to restore to its pristine state), of course you need access to the internet, and you will need a github token to access the OS-Climate hithub.

The **Sample Data template** provides both a *Read me* sheet and sheet of data dictionary *Definitions*, as well as three input data sheets:
* ITR input data: The fundamental financial, emissions, and production data of companies, listed by security instrument as company id
* ITR target input data: Short-term Emissions or Intensity reduction targets and Net-Zero attainment target dates, listed by company id
* Portfolio: A list of positions and investment value amounts

The user may choose **Benchmark Data** that forecasts intensity reductions expected from 2020-2050 by region and sector.  By default we use the OECM benchmark, but two TPI benchmarks are also available.  In all three cases we use the same projections for production grwoth forecasts.  We also use the same global carbon budget and TCRE multipliers for all benchmarks.

After scoring the portfolio, the portfolio is copied to the local file *data_dump.xlsx* which can be downloaded for further analysis.

This notebook also outputs an **enhanced portfolio** (with temperature scores), can be aggregated using various weighting methods to gain additional portfolio alignment insights.

Please enjoy learning how the ITR tool works by following the computations performed by this Jupyter Notebook!

## Getting started
Make sure you are running the notebook with the requirements installed available in the example folder.

If you see errors when attempting to load the ITR modules, go to the top-level ITR directory, activate the `itr_env` conda environment (using `conda activate itr_env` and execute the command `pip install -e .`.  Then try again, or hit the <i class="fas fa-forward"></i> button above.

In [ ]:
import os
import sys
import warnings

If all is well, the list of paths below will prioritize loading from the environment established for the ITR tool.  Please contact us if not.

In [ ]:
display(sys.path)

In [ ]:
# Load the ITR library files

import ITR
from ITR.data.excel import ExcelProviderProductionBenchmark, ExcelProviderIntensityBenchmark
from ITR.data.template import TemplateProviderCompany
from ITR.data.base_providers import BaseProviderProductionBenchmark, BaseProviderIntensityBenchmark
from ITR.data.data_warehouse import DataWarehouse
from ITR.portfolio_aggregation import PortfolioAggregationMethod
from ITR.temperature_score import TemperatureScore
from ITR.interfaces import ETimeFrames, EScope, IProductionBenchmarkScopes, IEIBenchmarkScopes
import pandas as pd

from ITR.data.osc_units import ureg, Q_, PA_

In [ ]:
print("Testing unit registry\n=====================")
one_co2 = ureg("CO2e")
print(f"The gas species CO2e, which was a gwp of 1: {one_co2}")

one_Gt_co2 = ureg("Gt CO2e")
print(f"A gigaton of CO2e: {one_Gt_co2}")

## Load the production and intensity benchmarks

Prepare the various benchmark files; the OECM benchmark is the default after the next cell finishes.

In [ ]:
import json

self_root = os.path.abspath('')
benchmark_prod_json = os.path.join(self_root, "data", "json-units", "benchmark_production_OECM.json")
benchmark_EI_OECM = os.path.join(self_root, "data", "json-units", "benchmark_EI_OECM.json")
benchmark_EI_TPI = os.path.join(self_root, "data", "json-units", "benchmark_EI_TPI_2_degrees.json")
benchmark_EI_TPI_below_2 = os.path.join(self_root, "data", "json-units", "benchmark_EI_TPI_below_2_degrees.json")

# load production benchmarks
with open(benchmark_prod_json) as json_file:
    parsed_json = json.load(json_file)
prod_bms = IProductionBenchmarkScopes.parse_obj(parsed_json)
base_production_bm = BaseProviderProductionBenchmark(production_benchmarks=prod_bms)

# load intensity benchmarks

# OECM
with open(benchmark_EI_OECM) as json_file:
    parsed_json = json.load(json_file)
ei_bms = IEIBenchmarkScopes.parse_obj(parsed_json)
OECM_EI_bm = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms)

# TPI
with open(benchmark_EI_TPI) as json_file:
    parsed_json = json.load(json_file)
ei_bms = IEIBenchmarkScopes.parse_obj(parsed_json)
TPI_EI_bm = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms)

# TPI below 2
with open(benchmark_EI_TPI_below_2) as json_file:
    parsed_json = json.load(json_file)
ei_bms = IEIBenchmarkScopes.parse_obj(parsed_json)
TPI_below_2_EI_bm = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms)

base_intensity_bm = OECM_EI_bm

## Download/load the sample template data

We have prepared sample data from public sources for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Template Requirements](https://github.com/os-c/ITR/blob/main/docs/DataTemplateRequirements.rst) section of the technical documentation for more details on data requirements and formatting. 

*The sample data may contain estimates, simplifications, and recategorizations.  It is intended to be generally representative, but not authoritative, and should not be relied upon to make investment decisions.*

In [ ]:
import urllib.request
from github import Github

# Create a Github instance with an access token.
# Use your shell's `export` command to inject your token into the GITHUB_TOKEN environment variable before starting this jupyter-lab instance.
gh = Github(os.environ['GITHUB_TOKEN'])

# Get repository by name and select the proper branch
repo = gh.get_repo("os-climate/ITR").get_branch(branch="develop-pint-steel-projections")

if not os.path.isdir("data"):
    os.mkdir("data")

for filename in ['data/20220415 ITR Tool Sample Data.xlsx',
                 'data/OECM_EI_and_production_benchmarks.xlsx',
                 'utils.py']:
    if not os.path.isfile(filename):
        # Get a specific content file:
        contents = repo.get_contents(f"examples/{filename}")

        # Donwnload file form ContenFile object info:
        urllib.urlretrieve(contents.download_url, filename)

try:  # Import statement when run in remote Jupyter servers from AWS Google etc..
    from utils import collect_company_contributions, plot_grouped_statistics, anonymize, \
        plot_grouped_heatmap, print_grouped_scores, get_contributions_per_group
except:  # Import statement when run locally
    from utils import collect_company_contributions, plot_grouped_statistics, anonymize, \
        plot_grouped_heatmap, print_grouped_scores, get_contributions_per_group

template_data_path = "data/20220415 ITR Tool Sample Data.xlsx"

### Logging
The ITR module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider for benchmark information and we are using the ITR Data Template for company and portfolio information, using *template_data_path* as the pathname to the data.  If you want to supply your own data file, you can set the pathname in the first line of the next cell.

In [ ]:
# Remove the # and space on the next line to point the template_data_path variable at your own data
# template_data_path = "data/your_template_here.xlsx"

template_company_data = TemplateProviderCompany(excel_path=template_data_path)

## Create the Data Warehouse

In [ ]:
template_provider = DataWarehouse(template_company_data, base_production_bm, base_intensity_bm)

# Fills in template_company_data._companies[0].projected_targets.S1S2

print(f"Benchmark Temperature = {base_intensity_bm.benchmark_temperature}\n\
Benchmark Global Budget = {base_intensity_bm.benchmark_global_budget}\n\
AFOLU included = {base_intensity_bm.is_AFOLU_included}")

## Load your portfolio

The portfolio data is a sheet in the Data Template named "Portfolio".

Please see the technical documentation in the [Data Template Requirements](https://github.com/os-c/ITR/blob/main/docs/DataTemplateRequirements.rst) section for details on data requirements.

In [ ]:
df_portfolio = pd.read_excel(template_data_path, sheet_name="Portfolio")
display(df_portfolio.tail())

To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://ofbdabv.github.io/ITR/autoapi/ITR/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [ ]:
companies = ITR.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the enhanced portfolio you'll find your original portfolio, with calculated temperature scores, trajectory and target scores and overshoot/undershoot ration, and a temperature_result which is current set to zero for all valid calculations.

In [ ]:
temperature_score = TemperatureScore(
    time_frames = [ETimeFrames.LONG],
    scopes=[EScope.S1S2],
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
enhanced_portfolio = temperature_score.calculate(data_warehouse=template_provider, portfolio=companies)

For every company the tool assigns a score for all the requested timeframe and scope combinations. For now the ITR methodolgy only supportt a long timeframe in combination with a S1S2 scope

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    display(enhanced_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']])

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. The temperature scores are calculated per time-frame/scope combination.


In [ ]:
aggregated_scores = temperature_score.aggregate_scores(enhanced_portfolio)
print(f"Temperature Score aggregation method = {temperature_score.aggregation_method}")

In [ ]:
aggregated_scores.long.S1S2.all.score

## Display aggregation data in various ways


The first analysis of your portfolio could be to understand if you have any particular hotspots in your portfolio. We can do that by analysing different groupings of companies.

The tool allows you to calculate temperature scores for _groups_ of companies. In this example we group the scores by sector and region, and leave the timeframe and scope unchanged. Any categorical variable in the data provided by you or your data provider (in the `test_company_data.xlsx` imported above) can be used as grouping variable, e.g. sectors, industries, regions, countries, market cap buckets, as well as the additional fields you imported via the portfolio data.

You can change the variable by which the data is grouped by replacing the fourth line in the following cell. For example, replacing "grouping=['sector', 'region']" by "grouping=['sector', 'country']" would result in temperature scores per sector per country. 
If you want to change the time frame as well, please refer to the section 'Calculate the company temperature scores' above.



In [ ]:
grouping = ['sector', 'region']
temperature_score.grouping = grouping
grouped_portfolio = temperature_score.calculate(data_warehouse=template_provider, portfolio=companies)
grouped_aggregations = temperature_score.aggregate_scores(grouped_portfolio)


The aggregated scores can then be used, for example, to show the relation between sectors and regions with respect to temperature score.
A visualization of this relation is shown in the heatmap below. The grey fields indicate that the portfolio contains no assest for those combinations.

#### Quick analysis: Heat Map

We can see here that our North American Steel is in reasonable shape. While Asian Steel can be improved as shown in the drill down below the graph



In [ ]:
analysis_parameters = ([ETimeFrames.LONG], [EScope.S1S2], grouping)
plot_grouped_heatmap(grouped_aggregations, analysis_parameters)

In [ ]:
region = 'Asia'
sector = 'Steel'
group = sector + '-' + region
group_contributions = get_contributions_per_group(grouped_aggregations, analysis_parameters, group)
group_contributions.round(2)

#### Sector analysis

Sector analysis may help us with bringing some clarity into our analysis from the heatmap above. In order to identify the companies that your portfolio would most benefit from engaging with, we can start with identifying the biggest contributing sectors to the portfolio's temperature score, as in our example below.

Contributions can be identified on an individual company level, as well as contributions from companies grouped by one of their characteristics, e.g. sector or region. The exact definitions of companies' contributions to the portfolio temperature scores depend on the selected aggregation method.

You can group companies on any categorical variable (e.g. sectors, countries, market cap buckets, investment strategies, etc)  you provide through your dataprovider, in this example in the test_company_data.xlsx imported above.

For our analysis we select one time-frame (LONG) and one scope (S1+S2) and group the outcomes on sector and compare AUM to temperature score contribution. We also then display the sector temperature scores.

#### Quick analysis

In this example we can see that both sectors Steel and Electricity are scoring above 2.0C. 


In [ ]:
time_frames = [ETimeFrames.LONG]
scopes = [EScope.S1S2]
grouping = ['sector']
analysis_parameters = (time_frames, scopes, grouping)

temperature_score = TemperatureScore(time_frames=time_frames,
                                     scopes=scopes,
                                     grouping=grouping)
enhanced_portfolio = temperature_score.calculate(data_warehouse=template_provider, portfolio=companies)
aggregated_portfolio = temperature_score.aggregate_scores(enhanced_portfolio)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    company_contributions = collect_company_contributions(aggregated_portfolio, enhanced_portfolio, analysis_parameters)

In [ ]:
plot_grouped_statistics(aggregated_portfolio, company_contributions, analysis_parameters)

We can use what we learned from this analysis to drill into a few of the highest contributing scoring sectors to see which
companies are the biggest contributors on the sector level. Considering the Steel sector, for instance, in the
table below.


In [ ]:
sector_contributions = company_contributions[['company_name', 'company_id', 'sector', 'contribution', 'temperature_score', 'ownership_percentage', 'portfolio_percentage']]
sector_contributions.loc[sector_contributions['sector'] == 'Steel'][:10].round(2)

### Save portfolio data for further analysis
To take your analysis outside of this notebook and for example for internal and client reporting, you can export all data to Excel and the clipboard for pasting into and analysing in other applications.

If you run the ITR tool locally or from Google Colab, you:
- Specify the filenames of the output files in the cell below, e.g. change 'data_dump.xlsx' in the first line to 'TS_output.xlsx'
- Run the cell below

In [ ]:
data_dump_filename = 'data_dump.xlsx'
enhanced_portfolio.set_index(['company_name', 'company_id']).to_excel(data_dump_filename)